In [ ]:
###########################
# Imports and Setup
###########################
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import plotly.express as px
import base64

# Configure OS routines
import os

# Configure data manipulation
import numpy as np
import pandas as pd

# Import CRUD module
from CRUD_Python_Module import AnimalShelter

# Import helper modules
from data_helpers import normalize_dataframe, bucket_categories
from rescue_filters import apply_rescue_filter
from dashboard_auth import validate_credentials, get_auth_error_message, is_authenticated


###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# Read all documents from database
df_raw = pd.DataFrame.from_records(db.read({}))

# Remove MongoDB _id column (ObjectID type causes issues with dash_table)
df_raw.drop(columns=['_id'], inplace=True)

# Normalize the dataframe (create age_weeks, sex, intact_status, valid_coords columns)
df = normalize_dataframe(df_raw)


###########################
# Dashboard Layout / View
###########################
app = JupyterDash(__name__)

# Load and encode logo
image_filename = 'Grazioso-Salvare-Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Authentication Layout
auth_layout = html.Div([
    html.Div([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                 alt='Grazioso Salvare Logo',
                 style={'height': '100px', 'display': 'block', 'margin': '20px auto'}),
        html.H2('Grazioso Salvare Dashboard Login',
                style={'textAlign': 'center', 'color': '#2c3e50'}),
        html.Hr(),
        html.Div([
            html.Label('Username:', style={'fontWeight': 'bold', 'marginBottom': '5px'}),
            dcc.Input(id='username-input', type='text', placeholder='Enter username',
                     style={'width': '100%', 'padding': '10px', 'marginBottom': '15px'}),
            html.Label('Password:', style={'fontWeight': 'bold', 'marginBottom': '5px'}),
            dcc.Input(id='password-input', type='password', placeholder='Enter password',
                     style={'width': '100%', 'padding': '10px', 'marginBottom': '15px'}),
            html.Button('Login', id='login-button', n_clicks=0,
                       style={'width': '100%', 'padding': '10px', 'backgroundColor': '#3498db',
                              'color': 'white', 'border': 'none', 'borderRadius': '5px',
                              'fontSize': '16px', 'cursor': 'pointer'}),
            html.Div(id='login-error', style={'color': 'red', 'marginTop': '15px', 'textAlign': 'center'})
        ], style={'maxWidth': '400px', 'margin': '0 auto', 'padding': '30px',
                  'backgroundColor': '#ecf0f1', 'borderRadius': '10px'})
    ], style={'padding': '50px'})
])

# Main Dashboard Layout
dashboard_layout = html.Div([
    # Header with logo and branding
    html.Div([
        html.A([
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                    alt='Grazioso Salvare Logo',
                    style={'height': '80px', 'display': 'block', 'margin': '0 auto'})
        ], href='https://www.snhu.edu', target='_blank'),
        html.H1('Grazioso Salvare Animal Rescue Dashboard',
               style={'textAlign': 'center', 'color': '#2c3e50', 'marginTop': '20px'}),
        html.P('Dashboard by Rick Goshen',
              style={'textAlign': 'center', 'fontStyle': 'italic', 'color': '#7f8c8d'}),
        html.P('CS 340 - Client/Server Development',
              style={'textAlign': 'center', 'color': '#95a5a6', 'fontSize': '14px'}),
    ], style={'padding': '20px', 'backgroundColor': '#ecf0f1', 'borderRadius': '10px', 'marginBottom': '20px'}),
    
    html.Hr(),
    
    # Filter Controls
    html.Div([
        html.H3('Select Rescue Type Filter:', style={'color': '#34495e'}),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': ' Water Rescue (Labrador, Chesapeake Bay Retriever, Newfoundland - Intact Female, 26-156 weeks)', 
                 'value': 'water'},
                {'label': ' Mountain/Wilderness Rescue (German Shepherd, Alaskan Malamute, Old English Sheepdog, Siberian Husky, Rottweiler - Intact Male, 26-156 weeks)', 
                 'value': 'mountain'},
                {'label': ' Disaster Rescue or Individual Tracking (Doberman Pinscher, German Shepherd, Golden Retriever, Bloodhound, Rottweiler - Intact Male, 20-300 weeks)', 
                 'value': 'disaster'},
                {'label': ' Reset (Show All Animals)', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'block', 'marginBottom': '10px'},
            style={'padding': '15px'}
        )
    ], style={'padding': '20px', 'backgroundColor': '#f8f9fa', 'borderRadius': '10px', 'marginBottom': '20px'}),
    
    html.Hr(),
    
    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        row_selectable='single',
        selected_rows=[0],
        sort_action='native',
        page_action='native',
        page_current=0,
        page_size=10,
        style_cell={
            'textAlign': 'left',
            'minWidth': '100px',
            'width': '150px',
            'maxWidth': '200px',
            'overflow': 'hidden',
            'textOverflow': 'ellipsis',
        },
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        },
        tooltip_data=[
            {
                column: {'value': str(value), 'type': 'markdown'}
                for column, value in row.items()
            } for row in df.to_dict('records')
        ],
        tooltip_duration=None
    ),
    
    html.Br(),
    html.Hr(),
    
    # Charts side-by-side
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

# Main App Layout with authentication state
app.layout = html.Div([
    dcc.Store(id='auth-state', data={'authenticated': False}),
    html.Div(id='page-content')
])


#############################################
# Interaction Between Components / Controller
#############################################

# Authentication callback
@app.callback(
    [Output('auth-state', 'data'),
     Output('login-error', 'children')],
    [Input('login-button', 'n_clicks')],
    [State('username-input', 'value'),
     State('password-input', 'value')]
)
def authenticate_user(n_clicks, username, password):
    """Handle user authentication."""
    if n_clicks == 0:
        return {'authenticated': False}, ''
    
    if validate_credentials(username, password):
        return {'authenticated': True}, ''
    else:
        error_msg = get_auth_error_message(username, password)
        return {'authenticated': False}, error_msg


# Page visibility callback (show login or dashboard based on auth state)
@app.callback(
    Output('page-content', 'children'),
    [Input('auth-state', 'data')]
)
def display_page(auth_state):
    """Display login page or dashboard based on authentication state."""
    if is_authenticated(auth_state):
        return dashboard_layout
    else:
        return auth_layout


# Filter callback - updates data table based on selected rescue type
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    """Filter the data table based on selected rescue type."""
    try:
        # Apply the appropriate filter
        filtered_df = apply_rescue_filter(df, filter_type)
        return filtered_df.to_dict('records')
    except Exception as e:
        print(f"Error in filter callback: {e}")
        # Return full dataset on error
        return df.to_dict('records')


# Chart callback - displays outcome type distribution
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    """Update the outcome type distribution chart based on filtered data."""
    if viewData is None or len(viewData) == 0:
        return html.Div([
            html.H4('No data to display', style={'textAlign': 'center', 'color': '#95a5a6'})
        ])
    
    dff = pd.DataFrame.from_dict(viewData)
    
    # Get outcome type counts
    outcome_counts = dff['outcome_type'].value_counts()
    
    # Apply category bucketing (top 10 + Other)
    category_mapping = bucket_categories(outcome_counts, top_n=10)
    dff['outcome_bucketed'] = dff['outcome_type'].map(category_mapping)
    
    # Create pie chart
    fig = px.pie(
        dff,
        names='outcome_bucketed',
        title='Outcome Type Distribution',
        color_discrete_sequence=px.colors.qualitative.Set3
    )
    
    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.update_layout(showlegend=True, height=500)
    
    return dcc.Graph(figure=fig)


# Style callback - highlights selected column
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    """Highlight selected columns in the data table."""
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# Map callback - updates geolocation map based on selected row
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    """Update the geolocation map based on selected row in data table."""
    if viewData is None or len(viewData) == 0:
        return html.Div([
            html.H4('No data to display on map', style={'textAlign': 'center', 'color': '#95a5a6'})
        ])
    
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]
    
    dff = pd.DataFrame.from_dict(viewData)
    
    # Validate coordinates exist and are valid
    if 'location_lat' not in dff.columns or 'location_long' not in dff.columns:
        return html.Div([
            html.H4('Location data not available', style={'textAlign': 'center', 'color': '#95a5a6'})
        ])
    
    # Get selected animal's coordinates
    try:
        lat = float(dff.iloc[row]['location_lat'])
        lon = float(dff.iloc[row]['location_long'])
        
        # Validate coordinate ranges
        if not (-90 <= lat <= 90 and -180 <= lon <= 180):
            raise ValueError("Coordinates out of range")
            
    except (ValueError, TypeError, KeyError):
        # Default to Austin, TX if coordinates are invalid
        lat, lon = 30.75, -97.48
    
    # Get animal details for marker
    breed = dff.iloc[row].get('breed', 'Unknown')
    name = dff.iloc[row].get('name', 'Unknown')
    
    return [
        dl.Map(
            style={'width': '100%', 'height': '500px'},
            center=[lat, lon],
            zoom=15,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[lat, lon],
                    children=[
                        dl.Tooltip(breed),
                        dl.Popup([
                            html.H4("Animal Name"),
                            html.P(name)
                        ])
                    ]
                )
            ]
        )
    ]


# Run the app
app.run(mode="inline", debug=False)